In [80]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set color map to have light blue background
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pathlib import Path
from ISLP import load_data

In [81]:
file_path = Path.cwd().joinpath("../../data/teacher-certification-and-years-of-experience.csv")
columns = [
    "C0",
    "State",
    "CT (FTE)",
    # "Classroom Teachers (FTE)",
    "MR (FTE)",
    # "Meeting All State Licensing/Certification Requirements (FTE)",
    "MR (P)",
    # "Meeting All State Licensing/Certification Requirements (P)",
    "FY (FTE)",
    # "Classroom Teachers in their First Year of Teaching (FTE)",
    "FY (P)",
    # "Classroom Teachers in their First Year of Teaching (P)",
    "SY (FTE)",
    "SY (P)",
    "schools",
    "schools (P)",
]

df = pd.read_csv(file_path, header=None,
                 names=columns,
                 )

df.drop(["C0"], axis=1, inplace=True)
# df = load_data(file_path)
# print(df.info())
# df.describe()
print(df.head())

           State    CT (FTE)    MR (FTE)  MR (P)  FY (FTE)  FY (P)  SY (FTE)  \
0  United States  3,138,535   3,084,697     98.3  216,088      6.9  163,067    
1        Alabama     47,203      46,839     99.2    2,744      5.8    2,296    
2         Alaska      7,707       7,639     99.1      507      6.6      305    
3        Arizona     56,969      54,072     94.9    4,565      8.0    4,047    
4       Arkansas     34,957      34,450     98.6    2,200      6.3    1,823    

   SY (P)  schools  schools (P)  
0     5.2  95,507         100.0  
1     4.9   1,397         100.0  
2     4.0     495         100.0  
3     7.1   1,913         100.0  
4     5.2   1,085         100.0  
